# Problem setting

In this tutorial, we demonstrate how graph neural networks can be used for recommendation. Here we focus on item-based recommendation model. This method in this tutorial recommends items that are similar to the ones purchased by the user. We demonstrate the recommendation model on the MovieLens dataset.

# Get started

DGL can be used with different deep learning frameworks. Currently, DGL can be used with Pytorch and MXNet. Here, we show how DGL works with Pytorch.

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

When we load DGL, we need to set the DGL backend for one of the deep learning frameworks. Because this tutorial develops models in Pytorch, we have to set the DGL backend to Pytorch.

In [ ]:
import dgl
from dgl import DGLGraph

# Load Pytorch as backend
dgl.load_backend('pytorch')

Load the rest of necessary libraries.

In [ ]:
import numpy as np
import pandas as pd
from scipy import stats
from scipy import sparse as spsp

# Dataset

We use the MoiveLens dataset for demonstration because it is commonly used for recommendation models. In this dataset, there are two types of nodes: users and movies. The movie nodes have three attributes: year, title and genre. There are ratings between user nodes and movie nodes. Each rating has a timestamp. In our recommendation model, we don't consider ratings and timestamps.

**Note**: It is not necessarily the best dataset to demonstrate the power of GNN for recommendation. We have prepared the dataset to simplify the demonstration.

To run the data preprocessing script, a user needs to download the English dictionary of the stanfordnlp package first. However, the following command only needs to run once.

In [ ]:
# Please uncomment the two commands when the tutorial is run for the first time.
#import stanfordnlp
#stanfordnlp.download('en')

Load the MovieLens dataset.

In [ ]:
from movielens import MovieLens
data = MovieLens('.')

Calculate some statistics of the dataset.

In [ ]:
ratings = data.ratings
user_id = np.array(ratings['user_idx'])
movie_id = np.array(ratings['movie_idx'])
user_movie_spm = spsp.coo_matrix((np.ones((len(user_id),)), (user_id, movie_id)))
num_users, num_movies = user_movie_spm.shape
print('#user-movie iterations:', len(movie_id))
print('#users:', num_users)
print('#movies:', num_movies)

We split the dataset into a training set and a testing set

In [ ]:
ratings_train = ratings[~(ratings['valid_mask'] | ratings['test_mask'])]
user_latest_item_indices = (
        ratings_train.groupby('user_id')['timestamp'].transform(pd.Series.max) ==
        ratings_train['timestamp'])
user_latest_item = ratings_train[user_latest_item_indices]
user_latest_item = dict(
        zip(user_latest_item['user_idx'].values, user_latest_item['movie_idx'].values))

Construct the training, validation and testing dataset

In [ ]:
# The training dataset
user_id = np.array(ratings_train['user_idx'])
movie_id = np.array(ratings_train['movie_idx'])
user_movie_spm = spsp.coo_matrix((np.ones((len(user_id),)), (user_id, movie_id)))
assert num_users == user_movie_spm.shape[0]
assert num_movies == user_movie_spm.shape[1]
train_size = len(user_id)
print('#training size:', train_size)

# The validation and testing dataset
users_valid = ratings[ratings['valid_mask']]['user_idx'].values
movies_valid = ratings[ratings['valid_mask']]['movie_idx'].values
users_test = ratings[ratings['test_mask']]['user_idx'].values
movies_test = ratings[ratings['test_mask']]['movie_idx'].values
valid_size = len(users_valid)
test_size = len(users_test)

# The recommendation model

At large, the model first learns item embeddings from the user-item interaction dataset and use the item embeddings to recommend users similar items they have purchased. To learn item embeddings, we first need to construct an item similarity graph and train GNN on the item graph.

There are many ways of constructing the item similarity graph. Here we use the [SLIM model](https://dl.acm.org/citation.cfm?id=2118303) to learn item similarity and use the learned result to construct the item graph. The resulting graph will have an edge between two items if they are similar and the edge has a weight that represents the similarity score.

After the item similarity graph is constructed, we run a GNN model on it and use the vertex connectivity as the training signal to train the GNN model. The GNN training procedure is very similar to the link prediction task in [the previous section](https://github.com/zheng-da/DGL_devday_tutorial/blob/master/BasicTasks_pytorch.ipynb).

## Construct the movie graph with SLIM
SLIM is an item-based recommendation model. When training SLIM on a user-item dataset, it learns an item similarity graph. This similarity graph is the item graph we construct for the GNN model.

Please follow the instruction on the [SLIM github repo](https://github.com/KarypisLab/SLIM) to install SLIM.

The SLIM only needs to run once and can be saved on the disk for future experiments.

In [ ]:
from SLIM import SLIM, SLIMatrix
model = SLIM()
params = {'algo': 'cd', 'nthreads': 2, 'l1r': 2.0, 'l2r': 1.0}
trainmat = SLIMatrix(user_movie_spm.tocsr())
model.train(params, trainmat)

Load the SLIM similarity matrix into DGL. We store the vertex similarity as edge data on DGL.

In [ ]:
movie_spm = model.to_csr()
print('#edges:', movie_spm.nnz)
print('most similar:', np.max(movie_spm.data))
print('most unsimilar:', np.min(movie_spm.data))

g = dgl.DGLGraph(movie_spm, readonly=True)
g.edata['similarity'] = torch.tensor(movie_spm.data, dtype=torch.float32)

Construct the item features.

In [ ]:
year = np.expand_dims(data.movie_data['year'], axis=1)
genre = data.movie_data['genre']
title = data.movie_data['title']
features = torch.tensor(np.concatenate((genre, title), axis=1), dtype=torch.float32)
print('#features:', features.shape[1])
in_feats = features.shape[1]

## GNN models

We run GNN on the item graph to compute item embeddings. In this tutorial, we use a customized [GraphSage](https://cs.stanford.edu/people/jure/pubs/graphsage-nips17.pdf) model to compute node embeddings. The original GraphSage performs the following computation on every node $v$ in the graph:

$$h_{N(v)}^{(l)} \gets AGGREGATE_k({h_u^{(l-1)}, \forall u \in N(v)})$$
$$h_v^{(l)} \gets \sigma(W^k \cdot CONCAT(h_v^{(l-1)}, h_{N(v)}^{(l)})),$$

where $N(v)$ is the neighborhood of node $v$ and $l$ is the layer Id.

The original GraphSage model treats each neighbor equally. However, the SLIM model learns the item similarity based on the user-item iteration. The GNN model should take the similarity into account. Thus, we customize the GraphSage model in the following fashion. Instead of aggregating all neighbors equally, we aggregate neighbors embeddings rescaled by the similarity on the edges. Thus, the aggregation step is defined as follows:

$$h_{N(v)}^{(l)} \gets \Sigma_{u \in N(v)}({h_u^{(l-1)} * s_{uv}}),$$

where $s_{uv}$ is the similarity score between two vertices $u$ and $v$.

The GNN model has multiple layers. In each layer, a vertex accesses its direct neighbors. When we stack $k$ layers in a model, a node $v$ access neighbors within $k$ hops. The output of the GNN model is node embeddings that represent the nodes and all information in the k-hop neighborhood.

<img src="https://github.com/zheng-da/DGL_devday_tutorial/raw/master/GNN.png" alt="drawing" width="600"/>

We implement the computation in each layer of the customized GraphSage model in `SAGEConv` and implement the multi-layer model in `GraphSAGEModel`.

In [ ]:
from sageconv import SAGEConv

class GraphSAGEModel(nn.Module):
    def __init__(self,
                 in_feats,
                 n_hidden,
                 out_dim,
                 n_layers,
                 activation,
                 dropout,
                 aggregator_type):
        super(GraphSAGEModel, self).__init__()
        self.layers = nn.ModuleList()

        # input layer
        self.layers.append(SAGEConv(in_feats, n_hidden, aggregator_type,
                                    feat_drop=dropout, activation=activation))
        # hidden layers
        for i in range(n_layers - 1):
            self.layers.append(SAGEConv(n_hidden, n_hidden, aggregator_type,
                                        feat_drop=dropout, activation=activation))
        # output layer
        self.layers.append(SAGEConv(n_hidden, out_dim, aggregator_type,
                                    feat_drop=dropout, activation=None))

    def forward(self, g, features):
        h = features
        for layer in self.layers:
            h = layer(g, h, g.edata['similarity'])
        return h

## Train Item Embeddings

We train the item embeddings with the edges in the item graph as the training signal. This step is very similar to the link prediction task in the [basic applications](https://github.com/zheng-da/DGL_devday_tutorial/blob/master/BasicTasks_pytorch.ipynb).

Because the MovieLens dataset has sparse features (both genre and title are stored as multi-hot encoding). The sparse features have many dimensions. To run GNN on the item features, we first create an encoding layer to project the sparse features to a lower dimension. 

In [ ]:
class EncodeLayer(nn.Module):
    def __init__(self, in_feats, num_hidden):
        super(EncodeLayer, self).__init__()
        self.proj = nn.Linear(in_feats, num_hidden)
        
    def forward(self, feats):
        return self.proj(feats)

We simply use node connectivity as the training signal: nodes connected by edges are similar, while nodes not connected by edges are dissimilar.

To train such a model, we need to deploy negative sampling to construct negative samples. A positive sample is an edge that exist in the original graph, while a negative sample is a pair of nodes that don't have an edge between them in the graph. We usually train on each positive sample with multiple negative samples.

After having the node embeddings, we compute the similarity scores on positive samples and negative samples. We construct the following loss function on a positive sample and the corresponding negative samples:

$$L = -log(\sigma(z_u^T z_v)) - Q \cdot E_{v_n \sim P_n(v)}(log(\sigma(-z_u^T z_{v_n}))),$$

where $Q$ is the number of negative samples.

With this loss, training should increase the similarity scores on the positive samples and decrease the similarity scores on negative samples.

In [ ]:
beta = 0
gamma = 0

class FISM(nn.Module):
    def __init__(self, user_movie_spm, gconv_p, gconv_q, in_feats, num_hidden):
        super(FISM, self).__init__()
        self.encode = EncodeLayer(in_feats, num_hidden)
        self.num_users = user_movie_spm.shape[0]
        self.num_movies = user_movie_spm.shape[1]
        self.b_u = nn.Parameter(torch.zeros(num_users))
        self.b_i = nn.Parameter(torch.zeros(num_movies))
        self.user_deg = torch.tensor(user_movie_spm.dot(np.ones(num_movies)))
        values = user_movie_spm.data
        indices = np.vstack((user_movie_spm.row, user_movie_spm.col))
        indices = torch.LongTensor(indices)
        values = torch.FloatTensor(values)
        self.user_item_spm = torch.sparse_coo_tensor(indices, values, user_movie_spm.shape)
        self.users = user_movie_spm.row
        self.movies = user_movie_spm.col
        self.ratings = user_movie_spm.data
        self.gconv_p = gconv_p
        self.gconv_q = gconv_q

    def _est_rating(self, P, Q, user_idx, item_idx):
        bu = self.b_u[user_idx]
        bi = self.b_i[item_idx]
        user_emb = torch.sparse.mm(self.user_item_spm, P)
        user_emb = user_emb[user_idx] / torch.unsqueeze(self.user_deg[user_idx], 1)
        tmp = torch.mul(user_emb, Q[item_idx])
        r_ui = bu + bi + torch.sum(tmp, 1)
        return r_ui
    
    def est_rating(self, g, features, user_idx, item_idx, neg_item_idx):
        h = self.encode(features)
        P = self.gconv_p(g, h)
        Q = self.gconv_q(g, h)
        r = self._est_rating(P, Q, user_idx, item_idx)
        neg_sample_size = len(neg_item_idx) / len(user_idx)
        neg_r = self._est_rating(P, Q, np.repeat(user_idx, neg_sample_size), neg_item_idx)
        return torch.unsqueeze(r, 1), neg_r.reshape((-1, int(neg_sample_size)))

    def loss(self, P, Q, r_ui, neg_r_ui):
        diff = 1 - (r_ui - neg_r_ui)
        return torch.sum(torch.mul(diff, diff)/2) \
            + beta/2 * torch.sum(torch.mul(P, P) + torch.mul(Q, Q)) \
            + gamma/2 * (torch.sum(torch.mul(self.b_u, self.b_u)) + torch.sum(torch.mul(self.b_i, self.b_i)))

    def forward(self, g, features, neg_sample_size):
        h = self.encode(features)
        P = self.gconv_p(g, h)
        Q = self.gconv_q(g, h)
        tot = len(self.users)
        pos_idx = np.random.choice(tot, int(tot/10))
        user_idx = self.users[pos_idx]
        item_idx = self.movies[pos_idx]
        neg_item_idx = np.random.choice(self.num_movies, len(pos_idx) * neg_sample_size)
        r_ui = self._est_rating(P, Q, user_idx, item_idx)
        neg_r_ui = self._est_rating(P, Q, np.repeat(user_idx, neg_sample_size), neg_item_idx)
        r_ui = torch.unsqueeze(r_ui, 1)
        neg_r_ui = neg_r_ui.reshape((-1, int(neg_sample_size)))
        return self.loss(P, Q, r_ui, neg_r_ui)

We evaluate the performance of the trained item embeddings in the item-based recommendation task. We use the last item that a user purchased to represent the user and compute the similarity between the last item and a list of items (an item the user will purchase and a set of randomly sampled items). We calculate the ranking of the item that will be purchased among the list of items.

In [ ]:
def RecEvaluate(model, g, features, users_eval, movies_eval, neg_sample_size):
    model.eval()
    with torch.no_grad():
        neg_movies_eval = data.neg_valid[users_eval].flatten()
        r, neg_r = model.est_rating(g, features, users_eval, movies_eval, neg_movies_eval)
        hits10 = (torch.sum(neg_r > r, 1) <= 10).numpy()
        print('HITS@10:{:.4f}'.format(np.mean(hits10)))
        return np.mean(hits10)

Now we put everything in the training loop.

In [ ]:
#Model hyperparameters
n_hidden = 16
n_layers = 1
dropout = 0.5
aggregator_type = 'sum'

# create GraphSAGE model
gconv_p = GraphSAGEModel(n_hidden,
                         n_hidden,
                         n_hidden,
                         n_layers,
                         F.relu,
                         dropout,
                         aggregator_type)

gconv_q = GraphSAGEModel(n_hidden,
                         n_hidden,
                         n_hidden,
                         n_layers,
                         F.relu,
                         dropout,
                         aggregator_type)

model = FISM(user_movie_spm, gconv_p, gconv_q, in_feats, n_hidden)


# Training hyperparameters
weight_decay = 5e-4
n_epochs = 100
lr = 1e-3
neg_sample_size = 20

# use optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)

# initialize graph
dur = []
prev_acc = 0
for epoch in range(n_epochs):
    model.train()
    loss = model(g, features, neg_sample_size)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    print("Epoch {:05d} | Loss {:.4f}".format(epoch, loss.item()))
    
    acc = RecEvaluate(model, g, features, users_valid, movies_valid, neg_sample_size)

print()
# Let's save the trained node embeddings.
RecEvaluate(model, g, features, users_test, movies_test, neg_sample_size)